# Tn-seq comparison of in vitro cultures of Mycobacterium tuberculosis and Mycobacterium bovis

Author: Jennifer J. Stiens

Code used in thesis chapter:
2.1 Identifying different gene requirements among the members of the MTBC

and publication: 

Gibson, A. J., Passmore, I. J., Faulkner, V., Xia, D., Nobeli, I., Stiens, J., Willcocks, S., Clark, T. G., Sobkowiak, B., Werling, D., Villarreal-Ramos, B., Wren, B. W., & Kendall, S. L. (2021). Probing Differences in Gene Essentiality Between the Human and Animal Adapted Lineages of the Mycobacterium tuberculosis Complex Using TnSeq. Frontiers in Veterinary Science, 8(December), 1–12. https://doi.org/10.3389/fvets.2021.760717

Step 1: 

Quality control of tn-seq reads

In [ ]:
#!/bin/bash

# iterate_fastqc.sh
# usage: bash iterate_fastqc.sh

#PATH={path to fastqc software}

FILES=*.fastq

for file in $FILES
do
	filename=$(basename "$file")
	filename="${filename%.*}"

	echo "File on the loop: 	$filename"

	#call fastQC quality analysis
	${PATH} ${file}

	echo -e "########################\n\n"
done


Step 2:
Create prot table for Mbovis (Mtb prot table available from TRANSIT)
TPP to assign reads to TA sites and generate .wig files
Look at statistics and examine distribution of insertions

In [ ]:
# create prot-table for Mbovis from .gff file
## try http:// if https:// URLs are not supported

#BiocManager::install("rtracklayer")  

library("rtracklayer")
library(GenomicRanges)

bovisTrack <- import("LT708304_updated_aug19.gff")

# use only relevant columns in dataframe
# (keep locus tag b/c 'gene' features don't have note)
bovis_df<-data.frame(descr = elementMetadata(bovisTrack)
  [,c("gene", "type", "note", "product", "locus_tag")],
  start = start(bovisTrack),
  end = end(bovisTrack), 
  strand = as.factor(strand(bovisTrack)))

# parse mbovis_df and put in new dataframe "prot_table"

prot_table <- data.frame(
  matrix(NA, ncol=9, nrow=nrow(bovis_df)), 
  stringsAsFactors = FALSE)

colnames(prot_table) <- c("PRODUCT","START", "END",
                          "STRAND", "AA_LEN", "TYPE", 
                          "GAP", "NAME", "ORF_ID")

for (i in 1:nrow(bovis_df)){
  # product, gene name, ORF id and aa len must be parsed from column 9 ()
  orf<-''
  gene_name<-''
  aa_len<-''
  if (is.na(bovis_df$descr.note[i])){
    note<-''
  }
  # if tRNA
  else if (bovis_df$descr.type[i]=="tRNA"){
    note<-unlist(strsplit(bovis_df[i,3],split = ","))
    orf<-note[1]
    gene_name <- note[1]

  }
  # if repeat region
  else if (bovis_df$descr.type[i]=="repeat_region"){
    note<-unlist(strsplit(bovis_df[i,3],split = ","))
    orf<-note[1]
    gene_name <- note[4]
  }
  else {
    note<-unlist(strsplit(bovis_df[i,3],split = ","))
    orf<-note[1]
    gene_name<- note[2]
    len<- substr(note[4], 7, nchar(note[4]))
    # get only integers from len string:
    x<- gregexpr("[0-9]+", len)
    aa_len <- as.numeric(unlist(regmatches(len, x)))
    #aa_len<- data[grep("[0-9]+", len),]
    }
  # product 
  if (!is.na(bovis_df$descr.product)[i]){
    prot_table$PRODUCT[i]<-bovis_df[i,4]
  }
  # start and end
  # make sure start and end in right order (could be switched?)
  if (bovis_df[i,7]<bovis_df[i,6]){
    prot_table$START[i]<-bovis_df[i,7]
    prot_table$END[i]<-bovis_df[i,6]
  } else {
    prot_table$START[i]<-bovis_df[i,6]
    prot_table$END[i]<-bovis_df[i,7]
  }
  
  # strand
  prot_table$STRAND[i]<-as.character(bovis_df$strand[i])

  # type
  if (!is.na(bovis_df$descr.type)[i]){
    prot_table$TYPE[i]<-as.character(bovis_df$descr.type[i])
  }
  # GAP
  prot_table$GAP[i]<-"-"
  
  # enter cells in dataframe if note is present:
  if (length(note)>0){
    # if type = Repeat, no aa len (nt length), 
    # in these is inputing '37' for aa_len (from H37Rv?)
    # aa_length
    if (length(aa_len)>0){
      prot_table$AA_LEN[i]<-aa_len
    }
    # gene name
    if (length(gene_name)>0){
      prot_table$NAME[i]<-gene_name
    }
    # orf
    if (length(orf)>0){
      prot_table$ORF_ID[i]<-orf
    }
  }
  # gene name if no note:
  else if (length(bovis_df$descr.gene[i])>0){
    prot_table$NAME[i]<-bovis_df$descr.gene[i]
  }
  # ORF id if no note (from locus-id)
  else {
    prot_table$ORF_ID[i] <- substr(bovis_df$descr.locus_tag[i], 
    8, 
    nchar(bovis_df$descr.locus_tag[i])
    )
  }
}
write.table( prot_table,
            "mbovis.prot_table",
            sep="\t", 
            col.names = F, 
            row.names = F, 
            quote = F
            )


In [ ]:
#!/bin/bash
# module load python/v3

PATH={path_to_tpp}
BWA_PATH={path_to_bwa}
FILES=*_1.fastq
fasta={path_to_genome_fasta}

for file in $FILES

do
        filename=$(basename "$file")
        filename="${filename%.*}"

        echo "File on the loop:         ${file}"
        
        tpp -bwa ${BWA_PATH} -ref ${fasta} -reads1 ${file} -output tpp_results/${filename}


done 

In [ ]:
#!/bin/bash

# establish quality of tpp data with transit tnseq_stats
# iterate_tnseq_stats.sh from inside file directory

FILES=*.wig

for file in $FILES
do

  filename=$(basename "$file")
  filename="${filename%.*}"

  echo "File on the loop: 	    ${filename}"
  echo "File on the loop:       ${file}"

  transit tnseq_stats ${file} -o ${filename}.dat
  
  echo -e "#####################################\n\n"
  
done

Step 3:

Essentiality analysis using TRANSIT HMM

In [ ]:
# make combined wig with 2 sub-libraries and no normalisation
transit export combined_wig bovis_hiseq_tpp_21.wig,bovis_hiseq_tpp_19.wig mbovis.prot_table combo_19_21.wig -n nonorm
# transit with hmm method and default TTR normalisation (sum because sparse datasets)
transit hmm -r Sum add_19_21.wig mbovis.prot_table hmm_bovis_add_19_21.wig


Essentiality analysis using Bio-Tradis (pipeline not appropriate for Himar1, so not used)

In [ ]:
module load python/v3
module load perl
#filter and remove tradis tags with Bio-Tradis
filter_tradis_tags -f  G_S3_L001_R1_001.fastq -t GTCTAGAGACCGGGGACTTATCAGCCAACCTGTTA -o G_S3.tag.fastq
remove_tradis_tags -f G_S3.tag.fastq -t GTCTAGAGACCGGGGACTTATCAGCCAACCTGTTA -o G_S3.rmtag.fastq
#assign to TA sites and calculate insertions (run in directory with fastqs)
bacteria_tradis -v -m 0 -f bovis_fastqs.txt -r Mbovis_AF2122_97.fasta
#determine essentiality calls
tradis_essentiality.R joined_output.m_bovis_BDG.csv

## Step 4

Resampling analysis using TRANSIT
Compares change in mean read count in genes between two datasets using custom prot table that compares only reciprocal blast/positionally homologous orthologs between two lineages with the same (+/- 1) number of TA sites. Bovis coordinates used with TB gene names to identify orthologs


In [ ]:
#resampling between mtb and mbovis

#use bovis on tb prot table which has bovis coordinates with tb names
#will show change between tb insertions to bovis insertions

cd ~/tn_seq/in_vitro_data
#!transit resampling in_vitro_data/perm_add_22_23.wig in_vitro_data/perm_add_19_21.wig in_vitro_data/mtbH37Rv.prot_table,in_vitro_data/bovis_on_tb_orthos.prot_table in_vitro_data/resampling_061123/bovis_on_tb_resamp_out.txt

#transit resampling perm_add_22_23.wig perm_add_19_21.wig mtbH37Rv.prot_table,bovis_on_tb_orthos.prot_table resampling_061123/bovis_on_tb_resamp_out_winz.txt -winz -PC 5
#transit resampling add_19_21.wig add_22_23.wig bovis_best.prot_table,tb_on_bovis.prot_table tb_on_bovis_resampling_output.txt

# with finding mean between reps instead of sum
transit resampling mtb_hiseq/TPP/mtb22_hiseq_tpp.wig,mtb_hiseq/TPP/mtb23_hiseq_tpp.wig bovis_hiseq/tpp/bovis_hiseq_tpp_19.wig,bovis_hiseq/tpp/bovis_hiseq_tpp_21.wig mtbH37Rv.prot_table,bovis_on_tb_orthos.prot_table resampling_061123/bovis_on_tb_resamp_mean_winz.txt -winz -PC 5 -s 100000


: 

Calculate mutational frequency of every TA site 

MF = insertion count at TA site / total insertion count

normalise data first with TTR

add pseudocount of 1 to each ta site (and to total ta sites) to avoid division by zero

In [ ]:
# make combined wig with 2 sub-libraries and TTR norm
!transit export combined_wig in_vitro_data/bovis_hiseq/tpp/bovis_hiseq_tpp_21.wig,in_vitro_data/bovis_hiseq/tpp/bovis_hiseq_tpp_19.wig ref_seqs/bovis_best.prot_table in_vitro_data/mf_analysis/norm_TTR_19_21.wig -n TTR
!transit export combined_wig in_vitro_data/mtb_hiseq/TPP/mtb22_hiseq_tpp.wig,in_vitro_data/mtb_hiseq/TPP/mtb23_hiseq_tpp.wig ref_seqs/Mtb_AL123456_3.prot_table in_vitro_data/mf_analysis/norm_TTR_2_23.wig -n TTR


In [5]:
# import with pandas and create columns for MF

import pandas as pd
import os

#read in wig file for bovis
df = pd.read_csv('~/tn_seq/in_vitro_data/mf_analysis/norm_TTR_19_21.wig', sep='\t', header=None, comment = "#")
#add column names
df.columns = ['bovis_ta_site', 'wig_21', 'wig_19', 'bovis_orf']
#sum of insertions at each site
df['mb_sum_ins'] = df['wig_21'] + df['wig_19']
#mean of insertions at each site
df['mb_mean_ins'] = df['mb_sum_ins']/2
#calculate total insertions for each dataset
#ins_19 = df['wig_19'].sum()
#ins_21 = df['wig_21'].sum()
total_ins_bovis = df['mb_sum_ins'].sum() + len(df)
print(total_ins_bovis)

#read in wig file for mtb
df2 = pd.read_csv('~/tn_seq/in_vitro_data/mf_analysis/norm_TTR_22_23.wig', sep='\t', header=None, comment = "#")
#add column names   
df2.columns = ['tb_ta_site', 'wig_22', 'wig_23', 'tb_orf']
#sum of insertions at each site
df2['tb_sum_ins'] = df2['wig_22'] + df2['wig_23']
total_ins_tb = df2['tb_sum_ins'].sum() + len(df2)
print(total_ins_tb)

ps_count_mb = total_ins_bovis/total_ins_tb
print(ps_count_mb)

#total_mean_ins = df['mean_ins'].sum()
#add one to each ta site to avoid zeros, (also add total number of ta sites to total_insertions)
df['mb_mf'] = ((df['mb_sum_ins'] + ps_count_mb) /total_ins_bovis) * 100
#calculate mf for each dataset and for summed insertions
#df['mb_mf'] = (df['mb_sum_ins'] /total_ins) * 100
#df['mf_21'] = df['wig_21']/ins_21
#df['mf_19'] = df['wig_19']/ins_19
#df['mf_mean'] = df['mean_ins']/total_mean_ins
#save to csv
df.to_csv('~/tn_seq/in_vitro_data/mf_analysis/bovis_mf.csv', index=False)


#mean of insertions at each site for tb
df2['tb_mean_ins'] = df2['tb_sum_ins']/2
#calculate mf from sum of insertions
df2['tb_mf'] = ((df2['tb_sum_ins'] + 1) /total_ins_tb) * 100
#save to csv
df2.to_csv('~/tn_seq/in_vitro_data/mf_analysis/mtb_mf.csv', index=False)

22848092.3
18915296.9
1.207916133740412


the pseudocount needs to be relative to the total insertion count. use ratio of bovis insertions/tb insertions as pseudocount to get readings at zero insertions equal



Compare log fold change in mutational frequency between two genomes. need to re-map ta sites from one genome to the other--using whole genome alignment?

alternative just gather for coding genes and compare TA sites within the coding gene can use gene names by ta sites and list of orthologs

In [7]:
#gene focussed analysis
#import gene name df
import pandas as pd
gene_names = pd.read_csv('~/tn_seq/in_vitro_data/mf_analysis/ortho_gene_names.csv', sep=',', header=0, comment = "#")

#remove extraneous gene names in mf spreadsheets
bovis_df = pd.read_csv('~/tn_seq/in_vitro_data/mf_analysis/bovis_mf.csv', sep=',', header=0)
tb_df = pd.read_csv('~/tn_seq/in_vitro_data/mf_analysis/mtb_mf.csv', sep=',', header=0)
bovis_df['bovis_orf'] = bovis_df['bovis_orf'].str.split(' ').str[0]
tb_df['tb_orf'] = tb_df['tb_orf'].str.split(' ').str[0]

#number the ta sites inside each gene
bovis_df["ordered_ta_site"] = bovis_df.groupby("bovis_orf").cumcount().add(1).astype(str)
tb_df["ordered_ta_site"] = tb_df.groupby("tb_orf").cumcount().add(1).astype(str)

#remove cols for individual wig files
bovis_df = bovis_df.drop(['wig_21', 'wig_19'], axis=1)
tb_df = tb_df.drop(['wig_22', 'wig_23'], axis=1)

#merge gene names with mf spreadsheets (only orthologs)
bovis_mf = pd.merge(gene_names, bovis_df, on='bovis_orf', how='left')
#print(bovis_mf.head())
#print(tb_df.head())


together_mf = pd.merge(bovis_mf, tb_df, left_on=('tb_orf', "ordered_ta_site"), right_on=('tb_orf', "ordered_ta_site"), how='left')
print(together_mf.head())
#complete_mf.to_csv('~/tn_seq/in_vitro_data/mf_analysis/together_mf.csv', index=False)

  bovis_orf  tb_orf  bovis_ta_site  mb_sum_ins  mb_mean_ins     mb_mf   
0    Mb0001  Rv0001           60.0         0.0          0.0  0.000005  \
1    Mb0001  Rv0001           72.0         0.0          0.0  0.000005   
2    Mb0001  Rv0001          102.0         0.0          0.0  0.000005   
3    Mb0001  Rv0001          188.0         0.0          0.0  0.000005   
4    Mb0001  Rv0001          246.0         0.0          0.0  0.000005   

  ordered_ta_site  tb_ta_site  tb_sum_ins  tb_mean_ins     tb_mf  
0             1.0        60.0         0.0          0.0  0.000005  
1             2.0        72.0         0.0          0.0  0.000005  
2             3.0       102.0         0.0          0.0  0.000005  
3             4.0       188.0         0.0          0.0  0.000005  
4             5.0       246.0         0.0          0.0  0.000005  


In [ ]:
# #find change in mf between datasets at each ta site
# import numpy as np
# complete_mf['mf_change'] = complete_mf['tb_mf'] - complete_mf['mb_mf']
# complete_mf['log2_mf_change'] = np.log2(abs(complete_mf['mf_change']))
# #change -inf to NA
# complete_mf['log2_mf_change'] = complete_mf['log2_mf_change'].replace(-np.inf, np.nan)
# #add column to indicate direction of change
# conditions = [ complete_mf['mf_change'] > 0, complete_mf['mf_change'] < 0, complete_mf['mf_change'] == 0 ]
# choices = [ 'up_tb', 'up_mb', 'no_change' ]
# complete_mf["change"] = np.select(conditions, choices, default=np.nan)
# print(complete_mf.loc[75:80,])
# complete_mf.to_csv('~/tn_seq/in_vitro_data/mf_analysis/complete_mf.csv', index=False)

In [8]:
# add in the transit hmm calls for each ta site
import pandas as pd
tb_calls = pd.read_csv("~/tn_seq/in_vitro_data/hmm_mtb_add_22_23.wig", sep="\t", comment="#", header=None, usecols=[0,6,7])
tb_calls.columns = ['tb_ta_site', 'tb_hmm_call', 'tb_orf']
tb_calls['tb_orf'] = tb_calls['tb_orf'].str.split('_').str[0]
bovis_calls = pd.read_csv("~/tn_seq/in_vitro_data/hmm_bovis_new_add_19_21.wig", sep="\t", comment="#", header=None, usecols=[0,6,7])
bovis_calls.columns = ['bovis_ta_site', 'bovis_hmm_call', 'bovis_orf']
bovis_calls['bovis_orf'] = bovis_calls['bovis_orf'].str.split('_').str[0]
bovis_calls['bovis_orf'] = bovis_calls['bovis_orf'].str.replace('MB', 'Mb')
#print(bovis_calls.head())

#merge with complete mf df
complete_mf = pd.merge(together_mf, tb_calls, on=('tb_orf', "tb_ta_site"), how='left')
complete_mf = pd.merge(complete_mf, bovis_calls, on=('bovis_orf', "bovis_ta_site"), how='left')
print(complete_mf.loc[75:80,])


   bovis_orf  tb_orf  bovis_ta_site  mb_sum_ins  mb_mean_ins     mb_mf   
75    Mb0003  Rv0003         3591.0         0.0         0.00  0.000005  \
76    Mb0003  Rv0003         3682.0         0.0         0.00  0.000005   
77    Mb0003  Rv0003         3708.0       505.3       252.65  0.002217   
78    Mb0003  Rv0003         3728.0         0.0         0.00  0.000005   
79    Mb0003  Rv0003         3739.0         8.7         4.35  0.000043   
80    Mb0003  Rv0003         3770.0         5.2         2.60  0.000028   

   ordered_ta_site  tb_ta_site  tb_sum_ins  tb_mean_ins     tb_mf tb_hmm_call   
75            13.0      3591.0         0.0         0.00  0.000005          NE  \
76            14.0      3682.0         0.0         0.00  0.000005          NE   
77            15.0      3708.0        97.1        48.55  0.000519          NE   
78            16.0      3728.0         0.0         0.00  0.000005          NE   
79            17.0      3739.0         0.0         0.00  0.000005          N

In [9]:

orig_cols = list(complete_mf.columns.values)
#print(orig_cols)
#print(complete_mf.shape)
new_cols = ['bovis_orf', 'tb_orf', 'bovis_ta_site', 'tb_ta_site', 'ordered_ta_site', 'bovis_hmm_call', 'tb_hmm_call', 'mb_sum_ins', 'tb_sum_ins', 'mb_mf', 'tb_mf']
new_complete_mf = complete_mf[new_cols]
#print(new_complete_mf.head(10))
#print(new_complete_mf.shape)
new_complete_mf.to_csv('~/tn_seq/in_vitro_data/mf_analysis/complete_mf.csv', index=False)

In [11]:
# #find change in mf between datasets at each ta site
import numpy as np
#copy complete mf df
change_mf = new_complete_mf.copy()
#add pseudocount of 1 to each mf value to avoid -inf
change_mf['fold_change'] = (new_complete_mf['mb_mf']  /(new_complete_mf['tb_mf'] ))
change_mf['log2_change'] = np.log2(change_mf['fold_change'])
#np.log2(new_complete_mf['tb_mf']) - np.log2(new_complete_mf['mb_mf'])
#change NaN to 0
#new_complete_mf['log2_change'] = new_complete_mf['log2_change'].replace(np.nan, 0)
print(change_mf.loc[75:80,])
change_mf.to_csv('~/tn_seq/in_vitro_data/mf_analysis/change_mf.csv', index=False)

   bovis_orf  tb_orf  bovis_ta_site  tb_ta_site ordered_ta_site   
75    Mb0003  Rv0003         3591.0      3591.0            13.0  \
76    Mb0003  Rv0003         3682.0      3682.0            14.0   
77    Mb0003  Rv0003         3708.0      3708.0            15.0   
78    Mb0003  Rv0003         3728.0      3728.0            16.0   
79    Mb0003  Rv0003         3739.0      3739.0            17.0   
80    Mb0003  Rv0003         3770.0      3770.0            18.0   

   bovis_hmm_call tb_hmm_call  mb_sum_ins  tb_sum_ins     mb_mf     tb_mf   
75             GD          NE         0.0         0.0  0.000005  0.000005  \
76             GD          NE         0.0         0.0  0.000005  0.000005   
77             NE          NE       505.3        97.1  0.002217  0.000519   
78             GD          NE         0.0         0.0  0.000005  0.000005   
79             GD          NE         8.7         0.0  0.000043  0.000005   
80             GD          NE         5.2         0.0  0.000028  0.0

## Run TRANSIT backwards for Mtb to compare the calls

In [1]:
#reverse the TA sites for the wig files for mtb
# transit with hmm method and default TTR normalisation (sum because sparse datasets) and reverse TA sites
#using same prot table but all TA sites in reverse order in wig file
transit hmm -r Sum add_22_23.wig Mtb_AL123456_3.prot_table reverse_transit/hmm_fwd_tb_add_22_23.wig
transit hmm -r Sum reverse_transit/add_22_23_rev.wig Mtb_AL123456_3.prot_table reverse_transit/hmm_rev_tb_add_22_23.wig


SyntaxError: invalid syntax (774125934.py, line 4)